In [2]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

import backtrader as bt
import akshare as ak
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

In [3]:
bb = ak.stock_zh_a_spot_em()

In [5]:
bb.columns

Index(['序号', '代码', '名称', '最新价', '涨跌幅', '涨跌额', '成交量', '成交额', '振幅', '最高', '最低',
       '今开', '昨收', '量比', '换手率', '市盈率-动态', '市净率', '总市值', '流通市值', '涨速', '5分钟涨跌',
       '60日涨跌幅', '年初至今涨跌幅'],
      dtype='object')

In [55]:
plt.rcParams["font.sans-serif"] = ["SimHei"]
plt.rcParams["axes.unicode_minus"] = False

In [22]:
import akshare as ak
import pandas as pd
from datetime import datetime

stock_hfq_df = ak.stock_zh_a_hist(symbol="600070", adjust="hfq", start_date='20000101', end_date='20050101').iloc[:, :6]
stock_hfq_df.columns = ['date','open','close','high','low','volume']
stock_hfq_df.index = pd.to_datetime(stock_hfq_df['date'])

start_date = datetime.strptime('20000101', '%Y%m%d')
end_date = datetime.strptime('20050101', '%Y%m%d')
data = bt.feeds.PandasData(dataname=stock_hfq_df, fromdate=start_date, todate=end_date)  # 规范化数据格式

In [67]:
len(data.lines.high)

1191

In [47]:
class TestStrategy(bt.Strategy):
    params = (
        ('maperiod', 15),
    )

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None

        # Add a MovingAverageSimple indicator
        self.sma = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.maperiod)

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # Not yet ... we MIGHT BUY if ...
            if self.dataclose[0] > self.sma[0]:

                # BUY, BUY, BUY!!! (with all possible default parameters)
                self.log('BUY CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()

        else:

            if self.dataclose[0] < self.sma[0]:
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()

In [49]:
cerebro = bt.Cerebro()
# Set initial cash to 100K
cerebro.broker.setcash(100000.0)
print(f'Starting Portfolio Value: {cerebro.broker.getvalue()}')
cerebro.adddata(data)
cerebro.addstrategy(TestStrategy)
cerebro.broker.setcommission(commission=0.001)
cerebro.addsizer(bt.sizers.FixedSize, stake=10)

cerebro.run()
print(f'Final Portfolio Value: {cerebro.broker.getvalue()}')

Starting Portfolio Value: 100000.0
2000-01-24, Close, 15.77
2000-01-24, BUY CREATE, 15.77
2000-01-25, BUY EXECUTED, Price: 15.93, Cost: 159.30, Comm 0.16
2000-01-25, Close, 15.65
2000-01-26, Close, 15.51
2000-01-27, Close, 15.21
2000-01-27, SELL CREATE, 15.21
2000-01-28, SELL EXECUTED, Price: 15.06, Cost: 159.30, Comm 0.15
2000-01-28, OPERATION PROFIT, GROSS -8.70, NET -9.01
2000-01-28, Close, 15.34
2000-01-28, BUY CREATE, 15.34
2000-02-14, BUY EXECUTED, Price: 16.27, Cost: 162.70, Comm 0.16
2000-02-14, Close, 17.13
2000-02-15, Close, 16.71
2000-02-16, Close, 16.82
2000-02-17, Close, 16.30
2000-02-18, Close, 16.97
2000-02-21, Close, 17.36
2000-02-22, Close, 16.58
2000-02-23, Close, 16.15
2000-02-24, Close, 16.33
2000-02-25, Close, 15.80
2000-02-25, SELL CREATE, 15.80
2000-02-28, SELL EXECUTED, Price: 15.80, Cost: 162.70, Comm 0.16
2000-02-28, OPERATION PROFIT, GROSS -4.70, NET -5.02
2000-02-28, Close, 16.12
2000-02-29, Close, 16.47
2000-02-29, BUY CREATE, 16.47
2000-03-01, BUY EXECUTED

In [8]:
df = ak.stock_zh_a_spot_em()[['代码', '名称']].head()
df = df[df['代码'].isin([])]

In [9]:
df

,代码,名称


ImportError: attempted relative import with no known parent package